In [12]:
import numpy as np
import pandas as pd
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from google.colab import drive

Information gathering from memepedia.ru/memoteka.

In [13]:
url = 'https://memepedia.ru/memoteka/'
response = requests.get(url).content

In [14]:
soup = BeautifulSoup(response, "html.parser")
soup_whole = soup.findAll(['body'])
#print(soup_whole)

In [15]:
name = []
view = []
date = []
rubric = []
url = []
img = []
description = []
origin = []
meaning = []

In [16]:
articles = soup.find_all('li', class_='post-item post-item-masonry-boxed')
#print(articles)

In [17]:
for article in articles:
  url.append(article.find('a').get('href'))
  view.append(article.find('span', class_="count").text)
  
  category_tmp = []
  for elem in article.find('div', class_="bb-cat-links").find_all('a'):
    category_tmp.append(elem.text)
  rubric.append(','.join(category_tmp))

  name.append(article.find('h2', class_="entry-title").text)

  img.append(article.find('div', class_="bb-media-placeholder").find('img').get('src'))

  response_tmp = requests.get(article.find('a').get('href')).content
  soup_tmp = BeautifulSoup(response_tmp, 'lxml')
  soup_whole_tmp = soup_tmp.find_all('body')[0]
  date.append(soup_whole_tmp.find('div', class_="author-info").find('time', class_="entry-date published ").text)
  description.append(soup_whole_tmp.find('div', class_="js-mediator-article s-post-content s-post-small-el bb-mb-el").find('p').text)
  soup_whole_tmp = soup_whole_tmp.find_all(['p', 'h2'])

  start_index = None
  end_index = None
  for elem in enumerate(soup_whole_tmp):
    if str(elem[1]) == '<h2>Происхождение</h2>':
      start_index = elem[0]
    elif str(elem[1]) == '<h2>Шаблон</h2>' or str(elem[1]) == '<h2>Галерея</h2>':
      end_index = elem[0]
      break

  sep_index = None
  for elem in enumerate(soup_whole_tmp[:end_index]):
    if str(elem[1]) == '<h2>Значение</h2>':
      sep_index = elem[0]

  tmp_list = []
  if sep_index != None:
    for line_index in range(start_index + 1, sep_index):
      tmp_list.append(soup_whole_tmp[line_index].text)
    origin.append(''.join(tmp_list))
    tmp_list = []
    for line_index in range(sep_index + 1, end_index):
      tmp_list.append(soup_whole_tmp[line_index].text)
    meaning.append(''.join(tmp_list))
  else:
    for line_index in range(start_index + 1, end_index):
      tmp_list.append(soup_whole_tmp[line_index].text)
    origin.append(''.join(tmp_list))
    meaning.append('None')

In [18]:
m = 5
print('URL', url[m])
print('VIEW', view[m])
print('RUBRIC', rubric[m])
print('NAME', name[m])
print('IMG', img[m])
print('DATE', date[m])
print('DESCRIPTION', description[m])
print('ORIGIN', origin[m])
print('MEANING', meaning[m])

URL https://memepedia.ru/krugi-na-polyax/
VIEW 1k
RUBRIC Тренды
NAME Круги на полях
IMG https://memepedia.ru/wp-content/uploads/2022/01/krugi-na-poljah-telegramm-1-360x240.jpg
DATE 31.01.2022 в 14:05
DESCRIPTION Круги на полях – телеграм-канал, который запустили комики ТНТ и стендаперы. Фишка проекта в том, что его участники общаются друг с другом с помощью видеосообщений (видеоответы, которые отображаются в кружочках).
ORIGIN Телеграм-канал “Круги на полях” запустился 17 января 2022 года. Всего за сутки его авторы набрали 70 тысяч подписчиков. Проект получил большой прирост за счет массового освещения в СМИ. А на девятый день существования канал пробил отметку в 700 тысяч подписчиков. На момент написания этой заметки у “Кругов” уже 840 тысяч подписчиков.Особенность канала в том, что его участники общаются с помощью видеоответов (так называемых “кружочков”). В чатах телеграма такая функция есть давно, но ранее никто не использовал ее для публичных проектов.Комики работают не по сценари

Inserting data into a table.

In [19]:
df = pd.DataFrame(data={'Название': name, 
                        'Просмотры': view, 
                        'Категории': rubric, 
                        'Дата публикации': date, 
                        'Описание': description, 
                        'Происхождение': origin, 
                        'Значение': meaning, 
                        'Ссылка на страницу': url, 
                        'Ссылка на картинку': img})
df

,Название,Просмотры,Категории,Дата публикации,Описание,Происхождение,Значение,Ссылка на страницу,Ссылка на картинку
0,Мистер Исключительный стареет,168,Видео,05.02.2022 в 12:25,Мистер Исключительный стареет (Твой любимый X)...,Мем со стареющим Мистером Исключительным – раз...,Мем со стареющим мистером Исключительным довол...,https://memepedia.ru/mister-isklyuchitelnyj-st...,https://memepedia.ru/wp-content/uploads/2022/0...
1,"Вставай, через не можу",5.6k,Видео,03.02.2022 в 12:17,"Вставай, через не можу (Вставай, через не могу...",Президент Украины Владимир Зеленский 31 декабр...,"Мем “Вставай, через не можу” показывает ситуац...",https://memepedia.ru/vstavaj-cherez-ne-mozhu/,https://memepedia.ru/wp-content/uploads/2022/0...
2,Родился ради,803,Видео,01.02.2022 в 16:46,Родился ради (Родиться ради) – постироничный м...,Мем “Родился ради…” является ответвлением мема...,Мем “Родиться ради” в ироничной форме показыва...,https://memepedia.ru/rodilsya-radi/,https://memepedia.ru/wp-content/uploads/2022/0...
3,Миньон едет на велосипеде,520,Видео,01.02.2022 в 15:17,Миньон едет на велосипеде (Minion rides a bike...,"Оригинальный эпизод, ставший мемом, взят из тр...","Мем, где миньон гоняет на велосипеде, иронично...",https://memepedia.ru/minon-edet-na-velosipede/,https://memepedia.ru/wp-content/uploads/2022/0...
4,Медведь пьет мочу,2.2k,"Пикча,Предложка",01.02.2022 в 12:34,Медведь пьет мочу (Медведь пьет фанту) – мем с...,"Мем основан на вирусном видео, на котором медв...",В мемах медведя со струей перерисовывают в раз...,https://memepedia.ru/medved-pet-mochu/,https://memepedia.ru/wp-content/uploads/2022/0...
5,Круги на полях,1k,Тренды,31.01.2022 в 14:05,"Круги на полях – телеграм-канал, который запус...",Телеграм-канал “Круги на полях” запустился 17 ...,None,https://memepedia.ru/krugi-na-polyax/,https://memepedia.ru/wp-content/uploads/2022/0...
6,Витя решай сам,11k,Видео,28.01.2022 в 15:14,Витя решай сам (Если ты хочешь со мной расстат...,Юля Хризантема – это пользовательница Тиктока ...,Мем “Витя решай сам” – это ироничный тренд в Т...,https://memepedia.ru/vitya-reshaj-sam/,https://memepedia.ru/wp-content/uploads/2022/0...
7,Люди меняются,1.1k,Пикча,26.01.2022 в 11:28,"Люди меняются – ироничный мем, показывающий, к...",Основой для мемов послужила вирусная картинка ...,Мем “Люди меняются” иронично сопоставляет люде...,https://memepedia.ru/lyudi-menyayutsya/,https://memepedia.ru/wp-content/uploads/2022/0...
8,Эй Богдан богом дан,16.9k,Видео,25.01.2022 в 12:47,Эй Богдан богом дан – мем со строчкой из песни...,Богдан – это детский образ стримера Дениса Сух...,Мем “Эй Богдан богом дан” – это ироничный трен...,https://memepedia.ru/ej-bogdan-bogom-dan/,https://memepedia.ru/wp-content/uploads/2022/0...
9,PIESEK (польская песня про песика),5.9k,Видео,24.01.2022 в 13:44,"PIESEK (Когда входишь в комнату, а там…) – тик...",Польский тиктокер @bulsjarz 10 декабря 2021 го...,Песни Piesek и Kotek – это милый залипательный...,https://memepedia.ru/piesek-kotek/,https://memepedia.ru/wp-content/uploads/2022/0...


In [20]:
drive.mount('/content/drive')
df.to_csv("/content/drive/My Drive/Colab Notebooks/memepedia_ru.csv", sep=',', index=False, encoding="utf-8")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
